***
**Downloading CSV and intializing imports**
===

In [ ]:
# #@title Download CSV
# #@markdown Supported URL types:
# #@markdown * Google Drive zip
# #@markdown * GitHub raw file

# csv_file_link = "https://github.com/codebasics/py/blob/master/ML/9_decision_tree/Exercise/titanic.csv" #@param {type:"string"}
# is_csv_from_github_comment = False #@param {type:"boolean"}

# import gdown
# import requests

# csv_name = ''

# if 'drive.google.com' in csv_file_link:
#   url = csv_file_link
#   url = url.replace('/file/d/', '/uc?id=').replace('/view?usp=sharing', '')
#   gdown.download(url, quiet=False)

# elif 'github.com' in csv_file_link and is_csv_from_github_comment is False:
#   url = csv_file_link.replace('github.com', 'raw.githubusercontent.com').replace('/blob', '')
#   response = requests.get(url)
#   content = response.content
#   output = url.split('/')[-1]
#   with open(output, 'wb') as f:
#       f.write(content)
#       print('[', output, ']', 'downloaded succesfully')
#       csv_name = output

# elif 'github.com' in csv_file_link and is_csv_from_github_comment is True:
#   url = csv_file_link
#   response = requests.get(url)
#   content = response.content
#   output = url.split('/')[-1]
#   with open(output, 'wb') as f:
#       f.write(content)
#       print('[', output, ']', 'downloaded successfully')
#       csv_name = output

In [ ]:
# df = pd.read_csv(csv_name)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import metrics
from sklearn import tree

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.impute import KNNImputer

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix, roc_curve, roc_auc_score
from sklearn.preprocessing import OneHotEncoder

In [ ]:
# * alternative df with more inputs (from 891 -> 1309)
df = pd.read_csv('titanic3.csv')

***
**Data Visualization**
===

In [ ]:
df.describe(include = 'all')

**Note:**

Keep in mind that the Percentage of people who survived the Titanic disaster is only 38.2% of the 1309±1 passengers who boarded the ship
see the chart below that includes the percentage of who survived and who died.

In [ ]:
deceased_percentage = (df['survived'] == 0).sum() / len(df) * 100
survival_percentage = (df['survived'] == 1).sum() / len(df) * 100
print(f"Deceased in percentage: {deceased_percentage:.1f}%")
print(f"Survived in percentage: {survival_percentage:.1f}%")

# Count the number of survivors and non-survivors
survival_counts = df['survived'].value_counts()

# Create a bar plot
plt.figure(figsize=(6, 4))
plt.bar(survival_counts.index, survival_counts.values, color=['tab:blue', 'tab:orange'])
plt.xlabel('Survival Status')
plt.ylabel('Count')
plt.title('Overall Survival')
plt.xticks([0, 1], ['Deceased', 'Survived'])
plt.show()


And as we can see here, people from class 1 are prioritized over class 2 and 3, the survivavility for class 2 and 3 are considerably lower than those in class one, the ratio of death is higher than those who survived.

meanwhile those in class 3 the mortality rate is higher than suvivability (500+ died, meanwhile only 180+ survived)

In [ ]:
# Create the bar plot
plt.figure(figsize=(8, 6))
df.groupby('pclass')['survived'].value_counts().unstack().plot(kind='bar')
plt.xlabel('Passenger Class')
plt.ylabel('Count')
plt.title('Survival Status by Passenger Class')
plt.legend(title='Survived', labels=['Deceased(0)', 'Survived(1)'])
plt.show()

Aside from people from class 1 are prioritized, it is believed that "WOMEN AND CHILDREN" are first to board the life rafts, which the figure below proves it
Women were prioritized to board the life boats since we can see that the bar chart below that the number of women who survived are greater than those who did not.

meanwhile the mortality rate for the men are greater than the survivability.

In [ ]:
# Create the bar plot
plt.figure(figsize=(8, 6))
df.groupby('sex')['survived'].value_counts().unstack().plot(kind='bar')
plt.xlabel('Sex type')
plt.ylabel('Count')
plt.title('Survival Status by Passenger Sex type')
plt.legend(title='Survived', labels=['Deceased(0)', 'Survived(1)'])
plt.show()

And like I've said earlier, "WOMEN AND CHILDREN" were prioritized to board the life rafts, so we can see here that CHILDREN have higher numbers of who survived and those who died.

In [ ]:
# creating new instance so it's separate case fromm the original DataFrame
df_visuals = df.copy()

In [ ]:
age_bins = [0, 10, 20, 30, 40, 50, 60, 70, 80]
age_labels = ['0-10', '10-20', '20-30', '30-40', '40-50', '50-60', '60-70', '70-80']

df_visuals['Age_Group'] = pd.cut(df_visuals['age'], bins=age_bins, labels=age_labels, right=False)

grouped_df = df_visuals.groupby(['Age_Group', 'survived'], observed=True).size().unstack(fill_value=0)

plt.figure(figsize=(8, 6))
grouped_df.plot(kind='bar', width=0.7)
plt.xlabel('Passenger Age')
plt.ylabel('Count')
plt.title('Survival Status by Passenger Age')
plt.legend(title='Survived', labels=['Deceased(0)', 'Survived(1)'])
plt.show()


In [ ]:
sibsp_bins = [0, 1, 2, 3, 4, 5, 6, 7, 8]
sibsp_labels = ['0-1', '1-2', '2-3', '3-4', '4-5', '5-6', '6-7', '7-8']

df_visuals['sibsp_Group'] = pd.cut(df_visuals['sibsp'], bins=sibsp_bins, labels=sibsp_labels, right=False)

grouped_df = df_visuals.groupby(['sibsp_Group', 'survived'], observed=True).size().unstack(fill_value=0)

plt.figure(figsize=(8, 6))
grouped_df.plot(kind='bar', width=0.7)
plt.xlabel('Sibling / Spouse')
plt.ylabel('Count')
plt.title('Survival Status by # of Passenger Sibling / Spouse')
plt.legend(title='Survived', labels=['Deceased(0)', 'Survived(1)'])
plt.show()


In [ ]:
fare_bins = [0,7.90,14.45,31.28,120]
fare_labels = ['Low','Mid','High_Mid','High']

df_visuals['fare_Group'] = pd.cut(df_visuals['fare'], bins=fare_bins, labels=fare_labels)

grouped_df = df_visuals.groupby(['fare_Group', 'survived'], observed=True).size().unstack(fill_value=0)

plt.figure(figsize=(8, 6))
grouped_df.plot(kind='bar', width=0.7)
plt.xlabel('Passenger Fare')
plt.ylabel('Count')
plt.title('Survival Status by Passenger Fare')
plt.legend(title='Survived', labels=['Deceased(0)', 'Survived(1)'])
plt.show()


***
**Data Preprocessing**
===

In [ ]:
sns.heatmap(df.isnull(), cbar = False).set_title("Missing values heatmap")

In this part of the code we can see there are many unique variables, the ones we'll likely use are the following:

TARGET VARIABLE:

SURVIVED

FEATURE VARIABLES:

PCLASS, SEX, AGE, SIBSP(?), FARE,(to be changed)

In [ ]:
df.nunique()

In [ ]:
# df.describe(include = 'all')

It is observed that there are a lot of missing values in our dataset, We'll clean them in the DATA CLEANING part
Target:
- delete the following: [name, ticket, cabin, boat, body, home.dest]
- use mean method for the following: [pclass, survived, sex, sibsp, parch, fare, ]

In [ ]:
print(df.isnull().sum())